In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data = pd.read_csv("C:/Users/amema/Downloads/bac-mauritanie-2022-predictive-modeling-challeng/train_set.csv")
test_data = pd.read_csv("C:/Users/amema/Downloads/bac-mauritanie-2022-predictive-modeling-challeng/test_set.csv", sep = ';')

In [3]:
import re
from datetime import datetime

# definir les noms des mois en arabe et en francais
arabic_months = ['يناير', 'فبراير', 'مارس', 'أبريل', 'مايو', 'يونيو', 'يوليو', 'أغسطس', 'سبتمبر', 'أكتوبر', 'نوفمبر', 'ديسمبر']
french_months = ['janv.', 'fév.', 'mars', 'avr.', 'mai', 'juin', 'juil.', 'août', 'sep.', 'oct.', 'nov.', 'déc.']


#  une expression réguliére pour extraire le jour, le mois et l'année séparément
arabic_regex = r'(\d{1,2})\s+(' + '|'.join(arabic_months) + r')\s+(\d{4})'
french_regex = r'(\d{1,2})\s+(' + '|'.join(french_months) + r')\s+(\d{4})'

# appliquer l'expression réguliére et créer un objet datetime
def convert_date(date_string):
    if re.search(arabic_regex, date_string):
        return datetime.strptime(re.search(arabic_regex, date_string).group(1)+' '+str(arabic_months.index(re.search(arabic_regex, date_string).group(2))+1)+' '+re.search(arabic_regex, date_string).group(3), '%d %m %Y')
    elif re.search(french_regex, date_string):
        return datetime.strptime(re.search(french_regex, date_string).group(1)+' '+str(french_months.index(re.search(french_regex, date_string).group(2))+1)+' '+re.search(french_regex, date_string).group(3), '%d %m %Y')
    else:
        return None

train_data['DateNaissance'] = train_data['DateNaissance'].apply(convert_date)
test_data['DateNaissance'] = test_data['DateNaissance'].apply(convert_date)

# convertir le format  datetime au format "YYYY-MM-DD" 
train_data['DateNaissance'] = train_data['DateNaissance'].dt.strftime('%Y-%m-%d')
test_data['DateNaissance'] = test_data['DateNaissance'].dt.strftime('%Y-%m-%d')


In [4]:
# Calculer la date de naissance  la plus fréquente
most_frequent_date = train_data['DateNaissance'].mode()[0]
most_frequent_date_ = test_data['DateNaissance'].mode()[0]


# Imputer les valeurs manquantes avec la date la plus fréquente
train_data['DateNaissance'] = train_data['DateNaissance'].fillna(most_frequent_date)
test_data['DateNaissance'] = test_data['DateNaissance'].fillna(most_frequent_date_)

In [5]:
from datetime import datetime

today = datetime.today()
today_2021 = today.replace(year=2021)
today_2022 = today.replace(year=2022)

# ajouter une nouvelle colonne avec la date d'aujourd'hui mais avec l'année 2021/2022 
train_data['Today2021'] = today_2021.strftime('%Y-%m-%d')
test_data['Today2022'] = today_2022.strftime('%Y-%m-%d')


# covertir la colonne 'DateNaissance' au format datetime
train_data['DateNaissance'] = pd.to_datetime(train_data['DateNaissance'])
test_data['DateNaissance'] = pd.to_datetime(test_data['DateNaissance'])


# calculer l'age de chaque etudiant
train_data['Age'] = (today_2021 - train_data['DateNaissance']) // pd.Timedelta(days=365.25)
test_data['Age'] = (today_2022 - test_data['DateNaissance']) // pd.Timedelta(days=365.25)


###  Random Forest Classifier

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier



# Sélection des caractéristiques (features) et la variable cible (target)
categorical_features = ['Etablissement', 'Serie,x','Centre','Willaya', 'moughataa']
numeric_features = ['Age']
target = 'Decision'

# Concaténation des ensembles d'entraînement et de test
all_data = pd.concat([train_data[categorical_features + numeric_features], test_data[categorical_features + numeric_features]], axis=0)



# Prétraitement des variables catégorielles
cat_transformer = OneHotEncoder(handle_unknown='ignore')
cat_transformer.fit(all_data)

# Encodage # Prétraitement des variables catégorielles
cat_transformer = OneHotEncoder(handle_unknown='ignore')
cat_transformer.fit(all_data[categorical_features])

# Prétraitement de la variable numérique
num_transformer = StandardScaler()
num_transformer.fit(all_data[numeric_features])

# Création du transformateur de colonnes des variables catégorielles de l'ensemble d'entraînement
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, categorical_features),
        ('num', num_transformer, numeric_features)
    ])

# Encodage des variables catégorielles de l'ensemble d'entraînement
X_train_encoded = preprocessor.fit_transform(train_data[categorical_features + numeric_features])

# Séparation des données en ensembles d'entraînement et de test encodés
X_test_encoded = preprocessor.transform(test_data[categorical_features + numeric_features])
y_train = train_data[target]

# Modèle Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Entraînement du modèle avec validation croisée
scores = cross_val_score(model, X_train_encoded, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Entraînement du modèle sur l'ensemble d'entraînement complet
model.fit(X_train_encoded, y_train)

# Faire la prédiction sur le nouveau jeu de données
y_pred = model.predict(X_test_encoded)

# Ajouter la prédiction en tant que colonne "Decision" dans le nouveau jeu de données
test_data['Decision'] = y_pred

# Ajouter la probabilité que la décision soit "Admis" dans le nouveau jeu de données
y_prob = model.predict_proba(X_test_encoded)
test_data['Predicted'] = y_prob[:,1]

data = test_data.loc[:, ['ID', 'Predicted']]

# Sauvegarde du nouveau jeu de données avec les prédictions et les ID
data.to_csv("C:/Users/amema/OneDrive/Desktop/DataChal/RandomForestC2.csv", index=False)


Accuracy: 0.80 (+/- 0.00)


In [7]:
count = (data['Predicted'] > 0.5).sum()
